# Centrality Algorithms

<a target="_blank" href="https://colab.research.google.com/github/neo4j/graph-data-science-client/blob/main/examples/centrality-algorithms.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This Jupyter notebook is hosted [here](https://github.com/neo4j/graph-data-science-client/blob/main/examples/centrality-algorithms.ipynb) in the Neo4j Graph Data Science Client Github repository.

Centrality algorithms are used to understand the role or influence of particular nodes in a graph. The notebook shows the application of centrality algorithms using the `graphdatascience` library on the Airline travel reachability network dataset that can be downloaded [here](https://snap.stanford.edu/data/reachability.html).

< TALK ABOUT TASKS >

### Setup

We start by importing our dependencies and setting up our GDS client connection to the database.

In [2]:
# Install necessary dependencies
#%pip install graphdatascience pandas

In [3]:
from graphdatascience import GraphDataScience
import pandas as pd
import os

C:\Users\kedar\anaconda3\envs\graph_stuff\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")
# NEO4J_AUTH = None
# if os.environ.get("NEO4J_USER") and os.environ.get("NEO4J_PASSWORD"):
#     NEO4J_AUTH = (
#         os.environ.get("NEO4J_USER"),
#         os.environ.get("NEO4J_PASSWORD"),
#     )

# gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

# # Replace with the actual connection URI and credentials
NEO4J_CONNECTION_URI = "bolt://XXXXXXXXXXXXX
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "XXXXXXXXXXXXX"

# Client instantiation
gds = GraphDataScience(
    NEO4J_CONNECTION_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)


### Importing the dataset

We import the dataset as a pandas dataframe first. 

In [6]:
df = pd.read_csv('https://snap.stanford.edu/data/reachability-meta.csv.gz', compression='gzip')
df.head()

,node_id,name,metro_pop,latitude,longitude
0,0,"Abbotsford, BC",133497.0,49.051575,-122.328849
1,1,"Aberdeen, SD",40878.0,45.459090,-98.487324
2,2,"Abilene, TX",166416.0,32.449175,-99.741424
3,3,"Akron/Canton, OH",701456.0,40.797810,-81.371567
4,4,"Alamosa, CO",9433.0,37.468180,-105.873599
